### Baseline

threshold on the dnbr? 

In [1]:
import rasterio as rio

In [2]:
img = rio.open('KENNETH_20250109.tif')

In [3]:
bands = img.read()

In [5]:
bands.shape

(5, 206, 239)

In [1]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import numpy as np

#### Implement a simple U-Net


* considering the image resolution
* different image size for each event
* split the dataset, avoid the leakage (spatially)

#### Other U-Net variants